# construct a contracted De Bruijn graph for tr-cross

By using '--label', we label the high degree nodes with the index of the sequences that contain them.

Note, this will also construct an mxt file containing the minhashes.

In [1]:
!rm -fr tr-cross
!../walk-dbg.py --label -x 1e7 -k 31 ../data/tr-cross.fa -o tr-cross


placing output in directory: tr-cross
gxt will be: tr-cross/tr-cross.gxt
mxt will be: tr-cross/tr-cross.mxt

building graphs and loading files
finding high degree nodes
... find_high_degree_nodes: 10000
... find_high_degree_nodes: 20000
... find_high_degree_nodes: 30000
... find_high_degree_nodes: 40000
... find_high_degree_nodes: 50000
... find_high_degree_nodes: 60000
... find_high_degree_nodes: 70000
... find_high_degree_nodes: 80000
... find_high_degree_nodes: 90000
... find_high_degree_nodes: 10000
... find_high_degree_nodes: 20000
... find_high_degree_nodes: 30000
... find_high_degree_nodes: 40000
... find_high_degree_nodes: 50000
... find_high_degree_nodes: 60000
... find_high_degree_nodes: 70000
... find_high_degree_nodes: 80000
... find_high_degree_nodes: 90000
... find_high_degree_nodes: 100000
traversing linear segments from 439 nodes
... 0 of 439
1315 segments, containing 213588 nodes
saving gxtfile tr-cross/tr-cross.gxt
saving mxtfile tr-cross/tr-cross.mxt
note: used/assi

# construct a catlas for tr-cross

`tr-cross.catlas.3.gxt` and `tr-cross.catlas.3.mxt` contain the catlas graph and minhashes.

`tr-cross.domgraph.3.gxt` is the graph made from the 3-dominating set.

`tr-cross.gxt` and `tr-cross.mxt` are the original input files.

`../spacegraphcats/gxt_to_gml.py` can be used to turn the domgraph or `tr-cross.gxt` files into GML files that can be visualized with Gephi.

In [2]:
!../build-catlas.py tr-cross 3

Project tr-cross in tr-cross
Found tr-cross.gxt in tr-cross
Loaded graph with 1315 vertices, 1315 edges and 1 components
Found tr-cross.mxt in tr-cross
Loaded minhashes for graph

Domset computation

Augmenting 1 2 3 

Catlas computation

Mapping graph vertices to dominators
Processing node 1315/1315
Building catlasses for connected components
Processing component 0/1
Catlas done
0 1
1 6
2 12
3 21
4 35
5 62
6 108
7 188


# calculate MinHash signatures from tr-1.fa and tr-2.fa

We know (by construction in this case) that tr-1.fa and tr-2.fa collectively cover the nodes in tr-cross.fa. Calculate MinHashes from each of them and use them to search, below.

Note, the MINHASH_MAX_SIZE can be modified below. the MINHASH_KSIZE must match what was used in walk-dbg.py, above; it defaults to k=31, and cannot go above 32 for now.

(These minhash signatures can be used to search the catlas .mxt file.)

In [3]:
from khmer import MinHash
import screed

MINHASH_MAX_SIZE=500   # important note: sensitivity in this parameter. not sure why. @CTB
MINHASH_KSIZE = 31

tr1_mh = MinHash(MINHASH_MAX_SIZE, MINHASH_KSIZE)
for record in screed.open('../data/tr-1.fa'):
    tr1_mh.add_sequence(record.sequence)

tr2_mh = MinHash(MINHASH_MAX_SIZE, MINHASH_KSIZE)
for record in screed.open('../data/tr-2.fa'):
    tr2_mh.add_sequence(record.sequence)
    
def dump_hashes_to_file(mh, filename):
    with open(filename, 'wt') as fp:
        hashes = mh.get_mins()
        hashes = map(str, hashes)
        fp.write(' '.join(hashes))

dump_hashes_to_file(tr1_mh, 'tr-cross/tr-1.mh.txt')
dump_hashes_to_file(tr2_mh, 'tr-cross/tr-2.mh.txt')

### side note - working with MinHashes and files

To load MinHashes from a file, create a new MinHash object and then use `MinHash.add_hash(h)` to add each hash from the file:

```
# load mh from a file
mh = MinHash(1000, 31)
with open(dumpfile, 'rt') as fp:
   hashes = fp.read().strip().split(' ')
   for h in hashes:
      h = int(h)
      mh.add_hash(h)
```

# Search the catlas with a minhash

In [4]:
# search for minhash of first sequence with label #1
!../search-for-domgraph-nodes.py tr-cross 3 tr-cross/tr-1.mh.txt 1

reading mxt file tr-cross/tr-cross.catlas.3.mxt
reading mh file tr-cross/tr-1.mh.txt
searching catlas minhashes w/tr-cross/tr-1.mh.txt
best match: similarity 0.972, catlas node 0
found 188 domgraph leaves under catlas nodes [0]
labels we're looking for: {1}
all labels: {1, 2}

pos dom nodes: 188
neg dom nodes: 0
all dom nodes: 188

tp: 95
fp: 93
fn: 0
tn: 0


In [5]:
# search for minhash of second sequence with label #2
!../search-for-domgraph-nodes.py tr-cross 3 tr-cross/tr-2.mh.txt 2

reading mxt file tr-cross/tr-cross.catlas.3.mxt
reading mh file tr-cross/tr-2.mh.txt
searching catlas minhashes w/tr-cross/tr-2.mh.txt
best match: similarity 1.000, catlas node 0
found 188 domgraph leaves under catlas nodes [0]
labels we're looking for: {2}
all labels: {1, 2}

pos dom nodes: 188
neg dom nodes: 0
all dom nodes: 188

tp: 96
fp: 92
fn: 0
tn: 0


## Some catveats

Things I will think about:

* right now I am only labeling nodes with degree > 2; linear segments are not labeled at all. Is this a problem?
